# Import

In [1]:
# Клонируем репозитерий stylegan2-ada-pytorch
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git /content/stylegan2-ada-pytorch

fatal: destination path '/content/stylegan2-ada-pytorch' already exists and is not an empty directory.


In [2]:
# Скачиваем модель StyleGAN2
!mkdir -p /content/StyleGAN2
!wget -q https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -O /content/StyleGAN2/ffhq.pkl # модель обученная на датасете ffhq(лица)

In [3]:
!pip install -q git+https://github.com/openai/CLIP.git # установка CLIP
!pip install -q ninja # утилита для сборки C++/CUDA кода
!pip install -q youtokentome # библиотека от Яндекса для работы с BPE-токенизатором

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
import sys
import os
sys.path.append('/content/stylegan2-ada-pytorch')
os.environ['TORCH_CUDA_ARCH_LIST'] = '7.5'

import pickle
import copy

import numpy as np

import clip

import torch
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image
from torchvision.transforms import transforms

from PIL import Image
import matplotlib.pyplot as plt

import re

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

RAND = 2

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.manual_seed(RAND)

In [4]:
# Загружаем свое фото
#image =Image.open('/content/kot.jpg')

#plt.figure(figsize=(8,8))
#plt.imshow(image)
#plt.axis('off')
#plt.show()

# Utils

In [3]:
def cleaned_text(text) -> str:
    """
    Простая очистка текста.
    """
    text = str(text) if text is not None else '' # преобразование text в строку, если это не строка
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s.,*!?:-]', '', text)  # удаление лишних символов (кроме пунктуации)
    text = re.sub(r'\s+', ' ', text).strip()  # удаление лишних пробелов
    text = re.sub(r'^[^\w]+', '', text)  # удаление пунктуации в начале строки
    text = text.strip(' .')  # убирает лишние пробелы и точки в начале и в конце строки
    return text

In [4]:
def preprocessing_text(model,
                       text: str,
                       device: torch.device) -> torch.Tensor:
  """
  Делает простую очистку текста, токенизирует и возвращает эмбеддинги.

  :param model: модель CLIP
  :param text: входной текст
  :param device: устройство для вычислений (CPU или CUDA)

  :return: эмбеддинг текста в пространстве CLIP
  """
  model.eval()
  with torch.no_grad():
    text = cleaned_text(text)
    text_tokenize = clip.tokenize(text).to(device)
    text_features = model.encode_text(text_tokenize)

  return text_features.detach()

In [5]:
def preprocessing_image(model,
                        preprocess_image,
                        image,
                        device: torch.device) -> torch.Tensor:
    """
    Если image - torch.Tensor, то преобразует его в PIL.
    Если image — PIL.Image, используется стандартный CLIP-препроцессор.
    Возвращает эмбеддинг изображения.

    :param model: модель CLIP
    :param preprocess_image: препроцессор модели CLIP для изображения
    :param image: входное изображение
    :param device: устройство для вычислений (CPU или CUDA)

    :return: эмбеддинг изображения в пространстве CLIP
    """
    model.eval()
    if isinstance(image, torch.Tensor):
      # Для torch.Tensor из StyleGAN2 с диапазоном [-1,1]
      transform = transforms.Compose([
          transforms.Lambda(lambda x: ((x + 1) / 2).clamp(0, 1)),
          transforms.Lambda(lambda x: F.interpolate(x,
                                                    size=(224, 224),
                                                    mode='bilinear',
                                                    align_corners=False)),
          transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                               std=(0.26862954, 0.26130258, 0.27577711))])

      image_preproc = transform(image.to(device))

      #image = ((image + 1) / 2).clamp(0, 1) # [-1, 1] → [0, 1]
      #image = (image * 255).to(torch.uint8) # конвертируем в диапазон [0, 255] и в тип uint8
      #image = image.permute(0, 2, 3, 1).cpu().numpy() # меняем формат с CHW → HWC и преобразуем в np.array
      #image = [Image.fromarray(img) for img in image] # cоздаём PIL.Image
      #image_preproc = torch.stack([preprocess(img) for img in image]).to(device)
    else:
      # Для PIL-изображений
      image_preproc = preprocess_image(image).unsqueeze(0).to(device)

    image_features = model.encode_image(image_preproc)

    return image_features

In [6]:
def make_latents(G,
                 RAND: int,
                 device: torch.device):
  """
  Герерирует латенты.

  :param G: генератор, используется для определение размера вектора
  :param RAND: фиксирует генератор случайных чисел
  :param device: устройство для вычислений (CPU или CUDA)

  :return: два латента (latent, latent_sample)
  """
  torch.cuda.manual_seed(RAND)
  latent_sample = torch.randn(4, G.z_dim, device=device)
  latent = torch.randn(2, G.z_dim, device=device)
  latent_plus = torch.randn(8, G.z_dim, device=device)
  return latent_sample, latent, latent_plus

# StyleGN2-ADA

In [7]:
with open('/content/StyleGAN2/ffhq.pkl', 'rb') as f:
  G = pickle.load(f)['G_ema'].to(device)

G_frozen = copy.deepcopy(G) # копия генератора (замороженный)

In [11]:
latent_sample, latent, latent_plus = make_latents(G,
                                                  RAND=RAND,
                                                  device=device)

c = None
G.eval()
image_gan = G(latent_sample,
              c,
              truncation_psi=0.7,
              noise_mode='const')

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [ ]:
img = ((image_gan +  1) / 2).clamp(0, 1)  # [-1, 1] → [0, 1]
grid = make_grid(img, nrow=2)
plt.figure(figsize=(8, 8))
plt.imshow(grid.permute(1, 2, 0).detach().cpu().numpy())
plt.axis('off')
plt.show()

# CLIP

In [8]:
# Загружаем модель и процессор
model, preprocess = clip.load('ViT-B/32', device=device)

In [ ]:
text_features = preprocessing_text(model, 'woman', device)

In [ ]:
image_features = preprocessing_image(model, preprocess, image_gan, device)

# Train

## Loss

### cosine_dist

In [9]:
def cosine_dist(image_features: torch.Tensor,
                text_features: torch.Tensor) -> torch.Tensor:
 """
 Вычисляет глобальный CLIP-loss (cosine distance).

 :param image_features: вектора изображения в пространстве CLIP
 :param text_features: вектора текста в пространстве CLIP

 :return: косинусное расстояние
 """
 global_loss = 1 - F.cosine_similarity(image_features, text_features, dim=1)
 return global_loss.mean()

### clip_loss

In [10]:
def delta_text(model,
               text_target: str,
               text_source: str,
               device: torch.device) -> torch.Tensor:
  """
  Вычисляет вектор текстового направления между таргетом и текущим стилем.

  :param model: модель CLIP
  :param text_target: текст целевого стиля
  :param text_source: текст исходного стиля
  :param device: устройство для вычислений (CPU или CUDA)

  :return: вектор текстового направления
  """
  emb_delta_text = preprocessing_text(model,
                                      text_target,
                                      device) - preprocessing_text(model,
                                                                   text_source,
                                                                   device)
  return emb_delta_text

In [11]:
def delta_image(model,
                preprocess_image,
                image: torch.Tensor,
                image_frozen: torch.Tensor,
                device: torch.device) -> torch.Tensor:
  """
  Вычисляет вектор визуального направления между изображением до и после fine-tuning.
  :param model: модель CLIP
  :param preprocess_image: препроцессор модели CLIP для изображения
  :param image: изображение из обучаемого генератора
  :param image_frozen: изображение из замороженного генератора
  :param device: устройство для вычислений (CPU или CUDA)

  :return: вектор визуального направления
  """

  emb_delta_image = preprocessing_image(model,
                                        preprocess_image,
                                        image,
                                        device) - preprocessing_image(model,
                                                                      preprocess_image,
                                                                      image_frozen,
                                                                      device)
  return emb_delta_image

In [12]:
def clip_loss(delta_image: torch.Tensor,
              delta_text: torch.Tensor) -> torch.Tensor:
  """
  Вычисляет направленный CLIP-loss.

  :param delta_image: вектор визуального направления
  :param delta_text: вектор текстового направления

  :return: направленный CLIP-loss (скаляр)
  """

  direction_clip_loss = 1 - F.cosine_similarity(delta_image, delta_text, dim=1)
  return direction_clip_loss.mean()

## Train

### cosine_dist

In [ ]:
def train_generator(generator,
                    model,
                    preprocess_image,
                    text: str,
                    epochs: int,
                    criterion,
                    device: torch.device,
                    RAND: int=2,
                    batch_size: int=2,
                    sample_size: int=4,
                    lr: float=0.0003,
                    c: torch.Tensor=None):
  """
  Обучает генератор StyleGAN2 для изменения изображений в направлении текста
  с использованием косинусного расстояния в качестве функции потерь.

  Функция выполняет fine-tuning генератора на фиксированном латентном векторе,
  вычисляет эмбеддинги изображения и текста через CLIP и минимизирует
  косинусное расстояние между ними, чтобы стилизовать изображение под заданный текст.

  Каждые 50 эпох сохраняет сгенерированные изображения.
  В конце визуализирует поведение функции потерь.

  :param generator: генератор StyleGAN2, который будет обучаться
  :param model: модель CLIP для получения текстовых и визуальных эмбеддингов
  :param preprocess_image: функция препроцессинга изображений для CLIP
  :param text: целевой текстовый промпт, определяющий желаемый стиль изображения
  :param epochs: количество эпох обучения генератора
  :param criterion: функция потерь, которая принимает текстовые и визуальные эмбеддинги и возвращает скалярный loss
  :param device: устройство для вычислений (CPU или CUDA)
  :param RAND: фиксирует генератор случайных чисел
  :param batch_size: размерность батча для обучения
  :param sample_size: размерность генерируемого сэмпла
  :param lr: скорость обучения для оптимизатора
  :param c: условие для генератора

  :return: None
  """
  loss_lst = []

  torch.cuda.manual_seed(RAND)
  os.makedirs('results', exist_ok=True)

  latent_sample = torch.randn(sample_size, generator.z_dim, device=device)
  latent = torch.randn(batch_size, generator.z_dim, device=device)

  optimizer = torch.optim.AdamW(generator.parameters(), lr=lr)

  generator.train()
  for param in generator.parameters():
    param.requires_grad = True

  for i in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    optimizer.zero_grad()

    image = generator(latent, c, truncation_psi=0.7, noise_mode='const')

    text_features = preprocessing_text(model,
                                       text,
                                       device)
    image_features = preprocessing_image(model,
                                         preprocess,
                                         image,
                                         device)

    loss = criterion(text_features, image_features)
    loss.backward()
    optimizer.step()

    loss_lst.append(loss.item())

    torch.cuda.empty_cache()

    print(f'Эпоха {i + 1}: loss={loss.item():.4f}')

    if i % 50 == 0:
      image_sample = generator(latent_sample,
                               c,
                               truncation_psi=0.7,
                               noise_mode='const')
      img = (image_sample.clamp(-1, 1) + 1) / 2.0  # [-1, 1] → [0, 1]
      grid = make_grid(img, nrow=2)

      save_path = f'results/generated_image_epoch_{i}.jpg'
      save_image(grid, save_path, 'jpeg')

      plt.figure(figsize=(8, 8))
      plt.imshow(grid.permute(1, 2, 0).detach().cpu().numpy())
      plt.axis('off')
      plt.show()

  plt.figure(figsize=(8, 8))
  plt.plot(loss_lst)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.title('График лосса')
  plt.grid(True)
  plt.show()

In [ ]:
train_generator(generator=G,
                model=model,
                preprocess_image=preprocess,
                text='werewolf',
                epochs=301,
                criterion=cosine_dist,
                device=device,
                RAND=2,
                batch_size=2,
                sample_size=4,
                lr=0.0003,
                c=None)

### clip_loss

In [13]:
def train_generator(generator,
                    generator_frozen,
                    model,
                    preprocess_image,
                    text_source: str,
                    text_target: str,
                    epochs: int,
                    criterion,
                    device: torch.device,
                    RAND: int=2,
                    batch_size: int=2,
                    sample_size: int=4,
                    batch_size_w: int=8,
                    lr: float=0.002,
                    k: int=12,
                    c: torch.Tensor=None):
  """
  Обучает генератор StyleGAN2 для изменения изображений в направлении текста
  с использованием CLIP-косинусного расстояния в качестве функции потерь.

  Функция выполняет fine-tuning генератора на фиксированном латентном векторе,
  вычисляет векторное направление между изображениями до и после обучения
  и текстовыми эмбеддингами, и минимизирует косинусное расстояние между ними.

  Каждые 50 эпох сохраняет сгенерированные изображения.
  В конце визуализирует график лосса.

  :param generator: генератор StyleGAN2, который будет обучаться
  :param generator_frozen: замороженный генератор, используемый для вычисления delta_image
  :param model: модель CLIP для получения текстовых и визуальных эмбеддингов
  :param preprocess_image: функция препроцессинга изображений для CLIP
  :param text_source: исходный текстовый промпт
  :param text_target: целевой текстовый промпт
  :param epochs: количество эпох обучения генератора
  :param criterion: функция потерь, которая принимает delta_image и delta_text и возвращает скалярный loss
  :param device: устройство для вычислений (CPU или CUDA)
  :param RAND: фиксирует генератор случайных чисел
  :param batch_size: размерность батча для обучения
  :param sample_size: размерность генерируемого сэмпла
  :param batch_size_w: размерность батча для оптимизации векторов из пространства W+
  :param lr: скорость обучения для оптимизатора
  :param k: количество слоев которые обучаются
  :param c: условие для генератора

  :return: None
  """
  loss_lst = []

  torch.cuda.manual_seed(RAND)
  os.makedirs('results', exist_ok=True)

  latent_sample = torch.randn(sample_size, generator.z_dim, device=device)
  latent = torch.randn(batch_size, generator.z_dim, device=device)
  latent_plus = torch.randn(batch_size_w, generator.z_dim, device=device)

  generator = generator.to(device)
  generator_frozen = generator_frozen.to(device)
  generator.train()
  generator_frozen.eval()

  lst_block = []
  for name, module in generator.synthesis.named_children():
    if name.startswith('b'):
        lst_block.append(name)

  for param in generator_frozen.parameters():
    param.requires_grad = False

  for epoch in tqdm(range(epochs)):

    # Оптимизация векторов из пространства W+
    w_plus_source = generator.mapping(latent_plus,
                                      c) # [batch_size, num_layers, dim]
    w_plus_target = generator.mapping(latent_plus,
                                      c).requires_grad_(True) # [batch_size, num_layers, dim]

    optimizer_w_plus = torch.optim.Adam([w_plus_target], lr=lr)

    optimizer_w_plus.zero_grad()

    image_w_plus = generator.synthesis(w_plus_target,
                                       noise_mode='const')
    image_w_plus_features = preprocessing_image(model,
                                                preprocess,
                                                image_w_plus,
                                                device)

    text_w_plus_features = preprocessing_text(model,
                                              text_target,
                                              device)

    loss_w_plus = cosine_dist(image_w_plus_features,
                              text_w_plus_features)

    loss_w_plus.backward()
    optimizer_w_plus.step()

    torch.cuda.empty_cache()

    delta_w_plus = w_plus_target.detach() - w_plus_source
    delta_norm = delta_w_plus.norm(p=2, dim=-1).mean(dim=0)

    topk_indices = torch.topk(delta_norm, k).indices

    # Заморозка mapping
    for param in generator.mapping.parameters():
      param.requires_grad = False

    # Заморозка synthesis
    for p in generator.synthesis.parameters():
      p.requires_grad = False

    # Заморозка affine и torgb параметров
    #for name, module in generator.synthesis.named_modules():
    #  lname = name.lower()
    #  if 'affine' in lname or 'torgb' in lname:
    #    for p in module.parameters():
    #      p.requires_grad = False

    # Заморозка всех слоев кроме топ-k
    for ind in topk_indices:
      block_idx = ind // 2
      conv_num = ind % 2

      block_name = lst_block[block_idx]
      block = getattr(generator.synthesis, block_name)

      if block_name == 'b4':
        block.conv1.weight.requires_grad = True
        block.conv1.bias.requires_grad = True
        #for p in conv.parameters():
        #    p.requires_grad = True
      else:
        if conv_num == 0:
          block.conv0.weight.requires_grad = True
          block.conv0.bias.requires_grad = True
        else:
          block.conv1.weight.requires_grad = True
          block.conv1.bias.requires_grad = True
        #for p in conv.parameters():
        #  p.requires_grad = True

    train_params = [param for param in generator.parameters() if param.requires_grad]
    optimizer = torch.optim.Adam(train_params, lr=lr)

    optimizer.zero_grad()

    image = generator(latent,
                      c,
                      truncation_psi=0.7,
                      noise_mode='const')
    image_frozen = generator_frozen(latent,
                                    c,
                                    truncation_psi=0.7,
                                    noise_mode='const')

    emb_delta_image = delta_image(model,
                                  preprocess_image,
                                  image,
                                  image_frozen,
                                  device)

    emb_delta_text = delta_text(model,
                                text_target,
                                text_source,
                                device)

    loss = criterion(emb_delta_image,
                     emb_delta_text) * 50

    loss.backward()
    optimizer.step()

    loss_lst.append(loss.item())

    torch.cuda.empty_cache()

    print(f'Эпоха {epoch + 1}: loss={loss.item():.4f}')

    if epoch % 50 == 0:
      image_sample = generator(latent_sample,
                               c,
                               truncation_psi=0.7,
                               noise_mode='const')
      img = (image_sample.clamp(-1, 1) + 1) / 2.0  # [-1, 1] → [0, 1]
      grid = make_grid(img, nrow=2)

      save_path = f'results/generated_image_epoch_{epoch}.jpg'
      save_image(grid, save_path, 'jpeg')

      plt.figure(figsize=(8, 8))
      plt.imshow(grid.permute(1, 2, 0).detach().cpu().numpy())
      plt.axis('off')
      plt.show()

  plt.figure(figsize=(8, 8))
  plt.plot(loss_lst)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.title('График лосса')
  plt.grid(True)
  plt.show()

In [ ]:
train_generator(generator=G,
                generator_frozen=G_frozen,
                model=model,
                preprocess_image=preprocess,
                text_source='human',
                text_target='werewolf',
                epochs=301,
                criterion=clip_loss,
                device=device,
                lr=0.002,
                c=None)